In [ ]:
!pip install fastapi
!pip install colabcode
# !pip install detoxify
!pip install transformers
!pip install pysentimiento

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from pysentimiento import create_analyzer
import time

In [ ]:
from transformers import BertTokenizer, AutoModelForSequenceClassification
#from transformers import RobertaTokenizer, RobertaModel
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-sentiment-analysis",output_attentions = False, 
        output_hidden_states = False).cuda()

#model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-sentiment-analysis").cuda()

In [ ]:
from fastapi import FastAPI
import pickle
import numpy as np
app = FastAPI()

In [ ]:
from colabcode import ColabCode
server = ColabCode(port = 10000, code = False)

In [ ]:
start = time.time()
temp_messages = ["sample message sample message sample message sample message sample message sample message  "]*1000

temp_1 = []
for i in temp_messages:
    temp = tokenizer(i,return_tensors="pt")
    temp_1.append(temp)

text = []
attention = []
for i in temp_1:
    text.append(np.array(i["input_ids"][0]))
    attention.append(np.array(i["attention_mask"][0]))

text = torch.IntTensor(text).cuda()
attention = torch.IntTensor(attention).cuda()
temp_model = None
with torch.no_grad():
    temp_model = model(text,attention)
print(temp_model.logits.shape)
print(temp_model.logits[:,0].shape)
print("Total time taken ", time.time() - start)

torch.Size([1000, 3])
torch.Size([1000])
Total time taken  0.22841787338256836


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  from ipykernel import kernelapp as app


In [ ]:
def get_predictions(input_message):
    input_message = input_message.split("-`!~")

    temp_1 = []
    for i in input_message:
        temp = tokenizer(i,return_tensors="pt",max_length = 130, truncation = True, padding = 'max_length')
        temp_1.append(temp)

    text = []
    attention = []
    for i in temp_1:
        text.append(np.array(i["input_ids"][0]))
        attention.append(np.array(i["attention_mask"][0]))


    text = torch.IntTensor(text).cuda()
    attention = torch.IntTensor(attention).cuda()

    #print(text,attention)
    
    with torch.no_grad():
        model_output = torch.nn.Softmax(dim = 1)(model(text,attention).logits)
    
    output = np.array(model_output.cpu()).mean(axis = 0)
    print(output)
    # output = []
    # for i in model_output:
    #     output.append(i.tolist())
    #print(output)
    output_dict = {"negative":float(output[0]),"neutral":float(output[1]),"positive":float(output[2])}
    return output_dict

def get_pred_single(message):
    pass

In [ ]:
get_predictions("String1-`!~String 2")

[0.03864501 0.81686497 0.14449003]


{'negative': 0.03864501044154167,
 'neutral': 0.8168649673461914,
 'positive': 0.14449003338813782}

In [ ]:
app = FastAPI()
@app.post('/abcd')
def index(string1):
    print(type(string1),string1)
    prediction = get_predictions(string1)
    print(prediction)
    return {'message':prediction}

server.run_app(app = app)

Public URL: NgrokTunnel: "https://6902-35-239-187-195.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [260]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


<class 'str'> !roulette all EZ Clap-`!~ACTION frostraider_16 won 6766 points in roulette and now has 13532 points! FeelsGoodMan-`!~!roulette 100-`!~ACTION potlug won 100 points in roulette and now has 250 points! FeelsGoodMan-`!~!roulette 2-`!~ACTION greenmonster653 won 2 points in roulette and now has 7 points! FeelsGoodMan-`!~!roulette all-`!~ACTION psycho_cat__ won 40767 points in roulette and now has 81534 points! FeelsGoodMan-`!~pokiL pokiL pokiL pokiL pokiL pokiL Amands2Mello pokiL pokiL pokiL pokiL pokiL pokiL-`!~mars youre really good at writing peepoGiggles-`!~@VenTMG 6:50pm-`!~!roulette 1-`!~!roulette 100-`!~peepoSad ModTime
[0.03811126 0.6643877  0.29750094]
{'negative': 0.038111262023448944, 'neutral': 0.6643877029418945, 'positive': 0.29750093817710876}
INFO:     168.5.40.83:0 - "POST /abcd?string1=%21roulette+all+EZ+Clap-%60%21~%01ACTION+frostraider_16+won+6766+points+in+roulette+and+now+has+13532+points%21+FeelsGoodMan%01-%60%21~%21roulette+100-%60%21~%01ACTION+p